# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [ ]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [ ]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPStatusError as e:
            # Handle 302 redirect manually if follow_redirects fails
            if e.response.status_code == 302 and "location" in e.response.headers:
                redirect_url = e.response.headers["location"]
                if not redirect_url.startswith("http"):
                    # handle relative redirects
                    redirect_url = urljoin(url, redirect_url)
                try:
                    response = await client.get(redirect_url, headers=headers)
                    response.raise_for_status()
                except Exception as e2:
                    print(f"Redirect request failed: {e2}")
                    return ""
            else:
                print(f"Request failed: {e}")
                return ""
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [5]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]
    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 인덕션 제품을 추천해 주세요. 각 제품의 주요 기능과 장점도 함께 알려 주세요.
Google Search Results: 10
Context 1: 카테고리
구매가이드
스토어
쇼핑리스트
회원가입 | 로그인
전기레인지
전기레인지
통합
노써치픽
랭킹
맞춤추천
탐색
구매가이드
인덕션 3구이하
인덕션 4구이상
하이브리드
Jason
업데이트
주방 가열기구는 가스에서 전기로 무게 중심이 옮겨지고 있고, 이 중 대세는 전자기 유도 방식의 인덕션 입니다. 가스레인지보다 2배이상 빠른 조리속도, 뜨겁지 않고 평평해 닦아내기 매우 편리한 점들 때문입니다.
전용 용기를 사용해야만 하는 단점에도 불구하고 전열조리기구의 75%이상을 인덕션이 점유하고 있습니다.
해외 제품이 주였던 초창기에는 생소한 방식과 비싼 가격 때문에 제품을 고르기 어려웠으나, 이제는 국내 기업을 중심으로 경쟁이 삼화되어 가격과 품질이 상당히 괜찮아 졌습니다.
가격에 가장 큰 영향을 끼치는 것은 화구 수인데 대화구1개+중소화구2개 조합의 3구 제품이 80~100만원 수준으로 성능과 가격의 밸런스가 좋아 가장 대중적입니다.
한편, 성능을 결정짓는 가장 중요한 요소는 출력입니다. 동시 최대출력과 더불어 개별 화구의 최대출력이 약하면 인덕션의 가장 큰 장점인 빠른 조리 속도를 십분 발휘할 수 없습니다.
이와 더불어 최근 다양하게 탑재 되고 있는 편의 기능까지 함께 고려해 가성비를 따져 5개 제품을 선정하였습니다.
더보기
🏆 공정하게 추천합니다!
제품협찬, 광고비 등 어떠한 대가도 받지 않았으며, 철저하게 소비자 입장에서 비교하여 추천합니다. 제휴링크를 통한 구매 시 노써치에 수익이 발생하지만, 추천제품의 선정과는 아무런 관계가 없습니다.
자세히 알아보기
🔍
핵심성능을 스펙과 후기

안녕하세요! 2025년 5월 기준, 삼성전자 3구 이상 인덕션 제품 중 추천할 만한 모델과 주요 기능 및 장점을 정리해 드리겠습니다.

---

## 1. 삼성전자 비스포크 AI 인덕션 NZ63DB607CF (3구 모델)

### 주요 특징
- **화력**: 대화구 3,400W, 중화구 2,600W, 소화구 1,200W로 상위 10% 이내의 최상급 화력
- **편의 기능**: 저소음(DCN) 기능, AI 넘침 방지 모드, 대화형 알림창, 1개의 심플하고 커진 조작부
- **디자인**: 프레임리스(테두리 프레임 없음), 얇아진 두께로 식기세척기와 설치 시 튀어나오는 부분 최소화
- **가격대**: 100만 원 미만으로 이전 비스포크 모델 대비 가격 경쟁력 크게 향상
- **상판 재질**: NEG 제품(프리미엄 SCHOTT 상판은 아님, 원가 절감)

### 장점
- 뛰어난 출력과 강력한 편의 기능을 갖춘 프리미엄급 인덕션
- 심플하고 세련된 디자인, 실용적인 AI 기능으로 안전과 사용 편의성 강화
- 합리적인 가격에 구매 가능해 가성비 우수

---

## 2. 삼성전자 비스포크 AI 인덕션 NZ63DB6506XW (상위 모델)

### 주요 특징
- NZ63DB607CF와 동일한 화력과 성능 보유
- 상판은 업계 최고 수준인 SCHOTT 제품 사용
- 스마트컨트롤 앱 지원으로 외부에서도 전원 제어 가능
- 가격은 픽 모델 대비 약간 높으나, 성능 대비 합리적

### 장점
- 최고급 상판과 스마트 기능 탑재로 내구성과 편의성 극대화
- 스마트폰으로 원격 제어 가능해 안전하고 편리함
- 프리미엄 인덕션 중에서도 뛰어난 가성비

---

## 3. 삼성전자 비스포크 인덕션 NZ63D650BXE (하위 모델)

### 주요 특징
- 화력은 픽 제품과 동일 (대화구 3,400W 수준)
- AI 끓음 감지 기능 없음, 조작부가 각 화구별 3개로 분리되어 있음
- 두께가 픽 제품보다 조금 더 두꺼움
- 가격대가 더 저렴해 예산 절약용 적합

### 장점
- 뛰어난 화력에 합리적인 가격 제공
- 편의 기능이 조금 부족하지만 기본 성능 충실
- 인덕션 입문용으로 가성비 우수

---

## 4. 삼성전자 비스포크 3구 인덕션 NZ63B4026AK (최고 가성비 모델)

### 주요 특징
- 3,400W 동급 출력 보유
- 상판은 블랙 컬러, 대화형 알림창 없음
- 가격대 55만 원~65만 원으로 매우 저렴함

### 장점
- 뛰어난 기본 화력과 삼성 품질 대비 저렴한 가격
- 색상이나 부가기능에 큰 관심 없으면 최고의 가성비
- 삼성 브랜드 신뢰도와 기본 성능 균형

---

## 5. 삼성전자 인덕션 프리미엄 케어 서비스

- 2023년 1월 1일 이후 구매 고객 대상
- **인덕션 코일 무상 보증 10년, 상판 무상 보증 3년**으로 연장
- 기타 부품은 1년 무상 보증 유지
- 소비자 신뢰 및 만족도 증대를 위한 프리미엄 서비스

---

# 요약 및 추천

| 모델명 | 화구 수 | 최대 출력(대화구) | 상판 재질 | 주요 편의 기능 | 가격대(원) | 특징 |
|--------|---------|--------------------|------------|-----------------|------------|------|
| NZ63DB607CF | 3구 | 3,400W | NEG | AI 넘침 방지, 저소음, 대화형 알림창 | 80만~100만 | 최상급 화력 + 뛰어난 편의성 + 합리적 가격 |
| NZ63DB6506XW | 3구 | 3,400W | SCHOTT | 스마트컨트롤, AI 끓음 감지 | 96만~115만 | 프리미엄 상판 + 스마트 기능 |
| NZ63D650BXE | 3구 | 3,400W | NEG | 기본 조작부, AI 끓음 감지 없음 | 66만~79만 | 기본 성능 + 저렴한 가격 |
| NZ63B4026AK | 3구 | 3,400W | NEG (블랙) | 기본 기능, 알림창 없음 | 55만~65만 | 최고의 가성비 모델 |

---

### 선택 가이드
- **최상급 화력과 최신 편의 기능, 합리적 가격**을 원하시면 **NZ63DB607CF** 추천
- **스마트 기능과 프리미엄 상판**을 중요하게 생각하면 **NZ63DB6506XW**가 좋음
- **예산을 절약하면서 기본 성능에 충실한 모델**은 **NZ63D650BXE** 또는 **NZ63B4026AK** 추천
- **보증 기간 연장 서비스**로 안심하고 오래 쓰실 수 있습니다.

---

필요하시면 더 자세한 모델별 비교나 구매처, 설치 안내도 도와드리겠습니다!

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV 모델을 추천해 주세요. 가격대와 주요 기능, 사용하기 편리한 점 등을 고려한 삼성전자 TV 추천을 부탁드립니다.
Google Search Results: 10
Request failed: 
Request failed: 
Context 1: 2020. 5. 17. 19:31
부모님댁 가전 선물하기 1탄​​​코드제로가 도착했음!무슨 청소기가 100만원이나..(인터넷 최저가 90만원...
m.blog.naver.com
부모님댁 가전 선물하기 3탄​​삼성전자 그랑데 드럼세탁기 (WF21N8750TW) 삼성전자 그랑데 건조기 (...
m.blog.naver.com...

--- End of Context ---

Context 2: 안녕하세요? 저는 사회초년생으로써 얼마 되지 않은 사람입니다.
부모님 집에 TV가 없어, 이번 기회에 성과급으로 TV를 놓아 드리려 합니다.
현재 가용가능한 재원은 많지 않지만 ~130만원 정도이고, 크기는 75인치 정도(거거익선을 생각해 최대 사이즈를 결정했습니다)로 생각하고 있습니다.
여러 영상이나 조사를 해 본 결과 삼성이나 LG, TCL에서 결정하고자 했고 각각 적합하다 싶은 후보모델을 아래와 같이 결정해 보았습니다.
(75C755때 놓친게 조금 아쉽네요)
부모님께서는 지금까지는 컴퓨터 모니터로 자연 다큐나 여행기 같은 영상을 많이 보셨기에 화질이 중요할 것 같아, 마음은 LED 보다 QLED인 TCL 쪽으로 많이 기울어서... 각 TV들을 실제 사용하신 분들이 있으시다면 실사용 경험을 듣고 추천받고자 첫 글을 올려 봅니다
다들 좋은 하루 되세요!
해당제품 사용하셨을 때 장점/단점이었다 하는 부분들도 있으셨다면 말씀해주시면 감사하겠습니다!ㅎㅎ
75c755 120만 이하로 자주 나오는 편인데 기다려보세요
비슷한 급인 하이센스 미니 l

안녕하세요! 2025년 5월 현재 기준으로 부모님께 선물하기 좋은 삼성전자 TV 모델을 추천드리겠습니다. 여러 검색 컨텍스트와 최신 정보를 반영해 가격대, 주요 기능, 사용 편리성 등을 고려해 안내해 드립니다.

---

## 1. 추천 모델 및 가격대

### 삼성 QLED 75형 모델 (예: QN75Q60B, QN75Q67B 등)
- **가격대:** 약 120만~130만원대 (시장 상황에 따라 변동 가능)
- **화질:** QLED 패널로 뛰어난 색재현력과 명암비 제공, 자연 다큐, 여행기 등 고화질 영상 감상에 적합
- **크기:** 75인치로 거실에 딱 좋은 대형 화면
- **가격 대비 성능:** 중상급 가격대에서 가성비 뛰어난 QLED 제품으로 부모님 시청 경험을 크게 향상

### 삼성 스마트 TV 43~55형 모델 (예: AU8000 시리즈 등)
- **가격대:** 50만~80만원대
- **주요 특징:** 스마트 기능 내장, 간편한 음성 인식, 리모컨 사용 편리, 인터넷 기반 콘텐츠 접근 용이
- **추천 이유:** 부담 없는 가격에 스마트 기능을 원하시는 경우 적합, 작은 공간에도 알맞음

---

## 2. 주요 기능 및 장점

- **화질 및 화면 크기:** QLED 기술로 일반 LED 대비 더 선명하고 생생한 색상 표현, 부모님 세대가 좋아하는 큰 화면으로 몰입감 제공
- **사용 편의성:** 삼성 스마트 리모컨은 직관적 버튼 배치와 음성 인식 지원으로 어르신들도 쉽게 조작 가능
- **삼성 스마트 허브:** 넷플릭스, 유튜브 등 인기 OTT 앱 내장으로 다양한 콘텐츠 간편 시청 가능
- **안정적인 A/S 및 브랜드 신뢰도:** 국내 대기업 삼성전자 제품으로 A/S가 빠르고 편리해 부모님께서도 안심하고 사용 가능

---

## 3. 구매 팁 및 고려 사항

- **할인 및 특가 이벤트:** 명절, 연말, 대형 할인 행사 때 가격이 내려가는 경우가 많으니 구매 시기를 잘 노리시는 게 좋습니다.
- **고객 후기 참고:** 삼성 QLED 시리즈는 소비자 만족도가 높고, TCL 같은 해외 브랜드 대비 국내 A/S와 브랜드 신뢰도가 우수해 부모님 세대에 적합합니다.
- **설치 공간 확인:** 75인치 TV는 큰 편이므로 설치 공간과 시청 거리(약 3~4m 권장)를 고려하세요.

---

## 요약

| 모델              | 가격대         | 화면 크기 | 주요 장점                             | 추천 대상                      |
|------------------|---------------|----------|----------------------------------|-----------------------------|
| 삼성 QLED 75형   | 120만~130만원 | 75인치   | 뛰어난 화질, 스마트 기능, 편리한 리모컨  | 고화질 영상 감상과 대화면 선호 시 |
| 삼성 스마트 TV 43~55형 | 50만~80만원  | 43~55인치 | 합리적 가격, 스마트 기능, 사용 편리       | 부담 없는 가격과 기본 기능 원할 때 |

---

필요하시면 최신 구매처, 세부 사양, 설치 서비스 관련 정보도 추가로 안내해 드리겠습니다. 부모님께서 편안하게 사용하시고 만족하실 수 있는 삼성 TV로 좋은 선물 되시길 바랍니다! 😊

감사합니다.

```markdown
삼성전자의 2025년 신제품은 2024년 제품에 비해 여러 면에서 개선 및 혁신이 이루어졌습니다. 주요 향상된 기능과 성능은 다음과 같습니다.

---

### 1. AI 통합 및 고도화
- **갤럭시 S25 시리즈**: 최신 AP와 향상된 NPU 성능으로 온디바이스 AI 기능이 강화되어, 전작 대비 통합적이고 상황과 맥락을 잘 이해하는 AI 경험을 제공합니다.
- **갤럭시 AI 플랫폼**: 멀티모달 인터랙션(터치, 제스처, 음성 명령) 지원으로 사용자와 더욱 직관적이고 자연스러운 소통이 가능해졌으며, 개인 맞춤형 인사이트 제공이 강화되었습니다.
- **AI 스마트폰 라인업 확대**: 플래그십뿐 아니라 중저가 제품군에도 AI 기능을 탑재해 전반적인 사용자 경험을 향상시켰습니다.

### 2. 반도체 및 메모리 기술
- **HBM3e 및 HBM4 개발 가속화**: 8단, 12단 HBM3e 제품을 양산 중이며, 2025년 하반기 HBM4 양산을 목표로 개발 중입니다. AI 및 HPC 수요에 대응하기 위한 고성능 메모리 공급 체계가 강화되었습니다.
- **DDR5, LPDDR5x 및 GDDR7 확대**: 서버용 DDR5, 모바일·PC용 LPDDR5x 고용량·고성능 제품 공급을 확대하고, GDDR7도 양산하여 AI와 HPC 응용처 대응력을 높였습니다.
- **메모리 시장 대응**: 선단 공정 전환 가속과 고부가가치 제품 비중 확대를 통해 경쟁력을 강화하며, 중국 업체의 레거시 제품 공급 확대에도 하이엔드 시장 집중 전략을 유지합니다.

### 3. 스마트폰 및 폴더블폰 혁신
- **폴더블폰 완성도 향상**: 슬림화, 경량화, 내구성 개선을 지속 추진하며, AI 경험을 폼팩터별로 특화하여 차별화된 사용성을 제공합니다.
- **신규 폼팩터 개발**: 사용자 경험과 품질을 최우선으로 고려한 신규 폼팩터를 준비 중이며, 시장 요구에 맞춘 혁신 제품 출시를 계획하고 있습니다.
- **프리미엄 및 중저가 전략**: 플래그십 제품은 고성능 AI 기능 강화에 집중하고, 중저가 시장에서도 AI 탑재와 핵심 사양 강화로 시장 점유율 확대를 추진합니다.

### 4. 디스플레이 기술 강화
- **저소비전력 OLED 기술**: AI 기능 확산에 맞춰 소비전력을 크게 낮춘 OLED 기술을 개발하고, 폴더블 등 차세대 폼팩터에 적용하여 디스플레이 경쟁력을 높였습니다.
- **8.6세대 IT OLED 라인 투자**: 노트북과 태블릿용 고화질 OLED 생산을 확대하며, 2026년 양산 목표로 차세대 라인업을 구축 중입니다.
- **QD-OLED 및 네오 QLED 확대**: 고해상도, 고주사율 프리미엄 제품군 확대와 초대형 TV 라인업 강화로 시장 리더십을 유지하고 있습니다.

### 5. 가전제품 AI 및 친환경 혁신
- **AI 기능 확대**: 냉장고, 세탁기 등 생활가전 전반에 AI 적용을 확대해 사용자의 편의성과 에너지 효율을 극대화하였으며, 음성 인식 및 맞춤형 서비스 제공을 강화했습니다.
- **친환경 기술 적용**: 2024년 출시된 갤럭시S25 시리즈에는 제조 과정에서 발생하는 폐배터리와 웨이퍼 트레일을 재활용한 소재를 사용해 자원순환과 환경 보호를 실천합니다.
- **비스포크 냉장고 신모델**: 디지털 컴프레서와 펠티어 소자 결합한 하이브리드 냉각 시스템으로 에너지 효율을 1등급 대비 최대 30% 향상시켰으며, 내부 용량과 수납 기능 개선, 자동 문 열림(오토오픈) 등 편의 기능도 강화되었습니다.

### 6. 로봇 및 신사업 투자
- **휴머노이드 로봇 개발 가속화**: 레인보우로보틱스 인수 및 미래로봇추진단 신설을 통해 AI와 메카트로닉스를 접목한 첨단 로봇 개발에 박차를 가하고 있습니다.
- **신사업 확장**: AI, 6G 등 미래 산업 분야 투자 확대와 고객 맞춤형 솔루션 개발로 신성장 동력을 모색하고 있습니다.

---

### 종합적 개선 사항
- **제품별 AI 최적화 및 기능 고도화**로 사용성, 개인화, 보안성을 강화
- **차세대 반도체와 메모리 기술 선도**로 AI 및 고성능 컴퓨팅 수요에 적극 대응
- **디스플레이 혁신과 친환경 기술 확대**로 제품 경쟁력과 지속 가능성 제고
- **신사업 및 로봇 분야 투자를 통한 미래 성장동력 확보**

---

### 참고
- 삼성전자는 2025년에도 AI와 혁신, 지속 가능한 경영을 핵심 비전으로 삼아 제품과 서비스에 첨단 기술을 접목하고 있으며, 고객 요구에 맞춘 프리미엄 제품 라인업과 차별화된 사용자 경험을 제공하는 데 주력하고 있습니다.
- 2025년 신제품은 2024년 대비 AI 통합, 반도체 고도화, 친환경 소재 활용, 폴더블 혁신, 디스플레이 및 가전 AI 기능 강화에서 큰 진전을 보이고 있습니다.

---

필요하시면 특정 제품군별 상세 기능이나 성능 개선점에 대해 더 알려드릴 수 있습니다.
```

...

삼성전자가 보유한 오디오 브랜드 중 JBL과 하만카돈(Harman Kardon)은 각각 독특한 특징과 강점을 가진 스피커 및 음향기기 브랜드입니다. 두 브랜드의 차이점을 기능, 음질, 디자인 측면에서 자세히 설명드리겠습니다.

---

## 1. 브랜드 개요 및 포지셔닝

- **JBL**
  - 하만 그룹의 대표적인 대중적 오디오 브랜드로, 다양한 사용자층을 대상으로 폭넓은 제품 라인업을 갖추고 있습니다.
  - 특히 휴대용 스피커, 홈 스피커, 프로페셔널 음향 장비 등에서 강세를 보이며, 활동적이고 트렌디한 감성을 반영한 제품이 많습니다.
  - 최신 모델은 빈티지, 레트로 스타일을 가미하여 재즈나 시티팝 같은 음악 장르에 최적화된 사운드를 제공합니다.

- **하만카돈 (Harman Kardon)**
  - 하만 그룹의 프리미엄 오디오 브랜드로, 세련되고 고급스러운 디자인과 균형 잡힌 고음질을 강조합니다.
  - 특히 홈 시어터, 하이파이 오디오 시장에서 강점을 가지며, 사운드의 정교함과 깊이를 중시하는 사용자에게 적합합니다.
  - 디자인적으로도 재활용 패브릭 소재와 알루미늄 등 고급 마감 소재를 사용하여 ‘쇠테리어’(금속 인테리어)와 같은 모던한 공간에 잘 어울립니다.

---

## 2. 음질 및 기능적 차이

| 구분       | JBL                         | 하만카돈                        |
|------------|-----------------------------|--------------------------------|
| **음질 특성** | 저음역대가 풍부하고 강력한 베이스, 재즈 등 특정 장르에 적합한 ‘재지한’ 사운드 특징<br>출력 약 100W 내외, 실내에서 풍부한 음량 제공 | 사운드 밸런스가 뛰어나고 고음과 저음의 해상력 우수<br>43Hz 초저역 재생, 자동 셀프 튜닝 기능 탑재로 어느 환경에서도 안정된 음향 제공 |
| **출력**    | 보통 100W 내외 (예: Authentics 300) | 고출력 160W (예: GO+PLAY 3)    |
| **채널**    | 2.0 스테레오 및 다채널 모델 다양 | 2.0 스테레오 중심                |
| **무선 기술** | 블루투스 5.3, Wi-Fi 지원 및 음성 명령 가능 | 블루투스 5.0, 자동 튜닝, USB-C 충전 등 포함 |
| **특징 기능** | 휴대성 강조(손잡이 포함), Wi-Fi 네트워크 지원, 음성 명령 기능 | 자동 셀프 튜닝, 패시브 라디에이터 내장, USB-C 충전 포트, 8시간 배터리 사용 가능 |

---

## 3. 디자인 및 인테리어 적합성

- **JBL**
  - 빈티지와 레트로 감성을 살린 디자인, 손잡이 등 휴대성을 고려한 실용적 요소 포함
  - 재즈나 시티팝 등 레트로 음악과 잘 어울리는 클래식한 외관
  - 내부 공간을 포인트 주는 인테리어에 적합

- **하만카돈**
  - 현대적이고 세련된 디자인, 알루미늄과 재활용 패브릭 사용으로 고급스러움 강조
  - ‘쇠테리어’와 같은 금속 소재 중심 인테리어에 조화로움
  - 스피커 위치에 따른 음향 조절 기능으로 공간 활용도가 높음

---

## 4. 요약 및 추천 포인트

| 항목     | JBL                                           | 하만카돈                                    |
|----------|-----------------------------------------------|---------------------------------------------|
| **주요 대상** | 재즈 및 레트로 음악 감상, 휴대성 중시 사용자    | 고음질과 세련된 인테리어, 균형 잡힌 사운드 추구자  |
| **음향 성능** | 저음 강조, 풍부한 베이스, 100W 출력 수준       | 균형 잡힌 고음질, 자동 튜닝, 160W 고출력         |
| **디자인**   | 빈티지, 손잡이 포함으로 실용적, 포인트 인테리어 | 알루미늄+패브릭, 모던하고 고급스러운 인테리어 적합 |
| **기능**    | Wi-Fi, 블루투스 5.3, 음성 명령, 휴대성 우수     | 블루투스 5.0, 자동 사운드 튜닝, USB-C 충전      |

---

## 참고

- 삼성전자와 하만의 시너지로 JBL과 하만카돈 모두 갤럭시 생태계와 연동성도 우수하여 삼성 스마트 기기 사용자에게 최적화된 경험 제공
- 각 브랜드별로 제품 라인업과 세부 모델별 특징이 다르므로, 사용 환경(휴대성, 인테리어, 음질 우선순위)에 맞춰 선택하는 것이 중요합니다.

---

필요하신 경우, 삼성전자 공식 홈페이지나 인증 판매점에서 최신 JBL 및 하만카돈 스피커의 상세 스펙과 가격 정보를 확인하실 수 있습니다.

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽만 새로 구매했는데, 한쪽 이어버드를 기존 기기와 페어링하는 방법을 자세히 알려주세요. 이어버드 한쪽 페어링 과정과 주의할 점도 포함해 주세요.
Google Search Results: 10
Context 1: ...

--- End of Context ---

Context 2: 안녕하세요 고객님,
안녕하세요 고객님!
로그인 하시면 보다 편리하게
서비스를 이용하실 수 있습니다.
무엇을 도와드릴까요?
기존에 연결된 이력이 있는 기기와는 한쪽 이어버드만으로 통화/스트리밍이 가능하나 새로운 기기와의 연결은 제한됩니다.
서비스센터에 방문 등을 통해 이어버드 구입 후에는 양쪽 이어버드 커플링 설정이 필요합니다.
1. 양쪽 이어버드를 충전 케이스 좌우 슬롯 홈에 맞게 넣어주세요.
2. 양쪽 이어버드를 동시에 7초 이상 길게 터치하세요.
3. 이어버드 배터리 상태 표시등이 녹색으로 깜빡인 후 꺼지면서 커플링이 완료됩니다.
※ 이어버드 커플링이 제대로 되지 않으면 배터리 상태 표시등이 빨간색으로 계속 깜빡입니다. 이어버드를 충전 케이스에서 뺏다가 다시 넣은 후 양쪽 이어버드를 동시에 7초 이상 길게 터치하세요..
이어버드 셀프 커플링 방법
갤럭시 버즈3 및 버즈3 프로의 이어버드는 아래 방법에 따라 커플링을 진행하세요.
1단계) 이어버드 한 쪽씩 충전 케이스에 넣고 충전 단자에 정상적으로 안착되었는지 확인하세요. 충전 케이스 LED가 녹색 또는 빨간색이 깜빡여야 합니다.
2단계) 충전 케이스를 열어둔 채로 케이스 아래의 충전 단자 옆 연결 버튼을 7초 이상 누르세요.
3단계) 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면서 커플링이 완료됩니다.
※ 충전 케이스 LED가 빨간색으로 계속 깜빡이는 경우, 단자 통신 오류로 1단계부터 다시 시도하세요. 정상 안착

안녕하세요! 갤럭시 버즈 이어버드 한쪽을 새로 구매하신 경우, 기존 이어버드와 정상적으로 커플링(페어링)하여 사용하려면 아래 방법을 참고해 주세요.

---

## 갤럭시 버즈 이어버드 한쪽 신규 구매 후 커플링(페어링) 방법

### 1. 커플링 전 준비사항
- 새로 구매한 이어버드와 기존 이어버드가 같은 모델 및 동일한 배터리 타입인지 확인하세요.
- 양쪽 이어버드의 소프트웨어 버전이 다르면 커플링이 불가능하므로, 필요 시 소프트웨어 업데이트가 필요합니다.
- 이어버드가 충분히 충전되어 있어야 합니다.

### 2. 이어버드 셀프 커플링 방법

#### (1) 이어버드 충전 및 안착
- 새 이어버드 한 쪽과 기존 이어버드 한 쪽을 충전 케이스의 좌우 슬롯 홈에 맞게 넣습니다.
- 충전 케이스 LED가 녹색 또는 빨간색으로 깜빡이는지 확인하여 정상적으로 충전 단자에 안착되었는지 확인하세요.

#### (2) 커플링 모드 진입
- 충전 케이스 뚜껑을 연 상태로, 케이스 아래 충전 단자 옆에 있는 연결 버튼을 7초 이상 길게 누릅니다.
- 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면 커플링이 완료된 것입니다.

#### (3) 휴대폰과 연결
- 휴대폰에서 Bluetooth 기능을 켜고, Galaxy Wearable(갤럭시 웨어러블) 앱을 실행합니다.
- 앱에서 갤럭시 버즈를 다시 연결하여 사용하세요.
- 만약 소프트웨어 버전이 다르면 앱에서 업데이트 팝업이 나타나니 "예"를 눌러 업데이트를 진행하세요.

---

## 주의 사항 및 팁

- **배터리 타입 일치 여부 확인:**  
  2019년 9월 생산분 이어버드를 분실한 후 2019년 10월 이후 제품을 구매할 때는 배터리 타입이 동일한지 반드시 서비스센터 방문 후 확인하세요. 배터리 타입이 다르면 커플링 작업이 불가능합니다.

- **커플링 실패 시 조치법:**  
  - 이어버드를 충전 케이스에서 빼냈다가 다시 넣고, 다시 7초 이상 동시에 터치하여 커플링을 시도하세요.  
  - 배터리 상태 표시등이 빨간색으로 계속 깜빡이면 접점 부위의 이물질, 부식, 케이스 침수 여부를 확인해야 합니다.  
  - 그래도 실패하면 서비스센터 방문을 권장합니다.

- **기기 초기화 및 재시도:**  
  커플링이 잘 안 될 때는 블루투스 기능을 껐다 켜거나, 갤럭시 웨어러블 앱에서 버즈를 초기화한 후 재연결해 보세요.

- **서비스센터 도움:**  
  서비스센터에 방문하시면 이어버드 구입과 커플링 작업을 엔지니어가 진행해드립니다. 특히, 양쪽 이어버드의 소프트웨어 버전이 많이 다르거나 커플링이 어려울 때 권장됩니다.

---

요약하자면, 새로 산 이어버드 한쪽과 기존 이어버드를 충전 케이스에 동시에 넣고 7초 이상 길게 누르면 커플링 모드로 진입하고, 이후 갤럭시 웨어러블 앱을 통해 연결과 소프트웨어 업데이트를 진행하시면 정상 사용이 가능합니다.

필요 시 언제든 삼성 서비스센터 방문을 통해 전문가의 도움을 받으실 수 있습니다.

궁금하신 점 있으면 언제든 문의해 주세요! 😊

---

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 S24 비교 차이
LLM Query: 삼성전자 S25 모델의 무게가 이전 모델인 S24와 비교했을 때 얼마나 차이나는지 알려 주세요. 두 모델의 무게 차이를 구체적으로 비교해 주세요.
Google Search Results: 10
Request failed: [SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] ssl/tls alert handshake failure (_ssl.c:1010)
Context 1: ...

--- End of Context ---

Context 2: 2025.05.13 (화)
[서울=뉴시스] 고범준 기자 = 23일 오후 서울 마포구 KT플라자 홍대애드샵플러스점에 삼성전자 갤럭시 S25 제품이 놓여 있다.
삼성전자 '갤럭시 S25 시리즈' 사전 판매는 오는 24일부터 2월 3일까지 진행하며 기본형과 플러스, 울트라 총 3개 모델로 내달 7일 국내에 공식 출시된다. 2025.01.23.
Copyright © NEWSIS.COM, 무단 전재 및 재배포 금지
많이 본 사진
결혼 후 더 예뻐진 조보아
사랑에 빠지는 남매
과감한 엄지원
컴백한 미야오
초능력소녀로 돌아온 이재인
한국 찾은 톰 아저씨
뉴시스Pic
그래픽뉴스
이시간
정치
국제
경제
금융
산업
IT·바이오
사회
문화
스포츠
연예
많이 본 기사
뉴시스 기획특집
높아지는 대출문턱
中 가전이 몰려온다
빌딩·상가 꿈틀
美관세 후폭풍
HBM이 구원투수
많이 본 기사
SKT, 효과 똑같다는 '유심 포맷' 시작했지만 현장선 '교체' 추천…왜?
체중감량 효과 누가 더?…'비만약' 위고비·마운자로, 충돌
역대 가장 얇은 AI폰 '갤S25 엣지' 베일 벗었다…149만원대
남규리, '완벽 몸매＋최강 동안' 미모…41세 맞아?
삼성전자, 초슬림 스마트폰 '갤럭시 S25 엣지' 출시 [뉴시스Pic]
[오늘의 주요일정]식품의약품안전처(5월13일 화

안녕하세요! 삼성전자 갤럭시 S25 모델과 이전 모델 S24의 무게 차이에 대해 구체적으로 비교해 드리겠습니다.

---

## 갤럭시 S25 vs S24 무게 비교

| 모델          | 무게       |
|---------------|------------|
| 갤럭시 S24    | 167g       |
| 갤럭시 S25    | 162g       |

- **무게 차이:** 약 5g 가벼워짐

---

## 상세 설명

- 갤럭시 S25 기본 모델은 **162g**으로, 이전 모델인 S24의 **167g**보다 **5g 가벼워졌습니다**.
- 경량화를 위해 소재 개선과 내부 부품 최적화를 진행하여 더 얇고 가벼운 구조로 설계되었습니다.
- 두께 역시 S24의 7.6mm에서 S25는 7.2mm로 0.4mm 더 얇아져 그립감과 휴대성이 향상되었습니다.
- 경량화 덕분에 장시간 사용 시 손목에 가해지는 부담이 줄어들어 사용자 경험이 좋아졌다는 평가가 많습니다.

---

## 참고

- S25 시리즈 중 플러스, 울트라 모델도 무게가 각각 더 가벼워졌습니다.
- 예를 들어, 울트라 모델은 218g으로 S24 울트라의 232g보다 14g 감소했습니다.

---

요약하자면, 갤럭시 S25는 S24보다 **5g 가벼운 162g**으로 설계되어 휴대성과 사용자 편의성을 높인 모델입니다. 이 경량화는 디자인과 소재 혁신에 기인하며, 보다 편안한 사용감을 제공합니다.

필요하시면 S25 시리즈 전체 모델별 무게 및 스펙 비교도 안내해 드릴 수 있습니다!